# simple tool to update .aws/credentials with user-prompted keys
Copy-and-paste from the keys generated by the Portal tool, e.g. copy this and paste into the prompt:

```
To update your AWS credentials file:

aws_access_key_id=ABCDEFGHIJK
aws_secret_access_key=987654321
aws_session_token=includetokenstringifitexists
```

Slightly smart, it that it ignores lines that do not start with one of the 3 keywords. Requires keywords be at start of line and no spaces around the '=' symbol, which is the default format for most key generators.

Puts the new keys at the top of the file with the label _[default]_, renames any previous 'default' key to _[prior-\<timestamp\>]_, then copies all existing entries below that so no keys are lost.

Will not update ~/.aws/credentials if both _aws_access_key_id_ and _aws_secret_access_key_ are not in the pasted input (_aws_session_token_ is optional as not all key generators create a token). 

You can look at the current default credentials at any time with the 'show_default_creds()' routine.


In [ ]:
# simple tool to update .aws/credentials with user-prompted keys
import time
import os

def update_credentials(ele):
    if ele == None or "aws_access_key_id" not in ele or "aws_secret_access_key" not in ele:
        print("No valid credentials found, exiting")
        return
    home=os.path.expanduser('~')
    with open(f"{home}/.aws/credentials","r") as fin:
        lines=fin.readlines()
    with open(f"{home}/.aws/credentials","w") as fout:
        date=time.time()
        fout.write(f"[default]\naws_access_key_id={ele['aws_access_key_id']}\n")
        fout.write(f"aws_secret_access_key={ele['aws_secret_access_key']}\n")
        if ele["aws_session_token"] != None:
               fout.write(f"aws_session_token={ele['aws_session_token']}\n")
        fout.write("\n")
        for line in lines:
            if line.startswith("[default]"):
                line = f"[prior-{date}]\n"
            fout.write(line)
    print("~/.aws/credentials updated with new [default] credentials.")

def show_default_creds():
    home=os.path.expanduser('~')
    with open(f"{home}/.aws/credentials","r") as fin:
        lines = fin.readlines()
    for i in range(5):
        print(lines[i],end="")
    
def ingest_userpaste():
    print("Expects cut-and-paste format of:\n \
    \taws_access_key_id=abcdefghi\n \
    \taws_secret_access_key=987654321\n \
    \taws_session_token=ttttttttt")

    raw = input()
    lines = raw.split(" ")
    ele = {}
    ele["aws_session_token"] = None # default in case it does not exist
    for line in lines:
        try:
            key, word = line.split("=")
            ele[key]=word
        except:
            pass
    if "aws_access_key_id" in ele and "aws_secret_access_key" in ele:
        return ele
    else:
        return None

In [ ]:
ele = ingest_userpaste()
update_credentials(ele)
show_default_creds()